# r/CasualConversations Database creation
This Jupyter Notebook is meant to create a database from submissions to r/CasualConversations. It first extracts submissions and then saves them into a special database file such that feature can be created from the submissions. This notebook is part of a project that tries to predict the flair of a submission on CasualConversations.

In [1]:
import praw # Source code : https://github.com/praw-dev/praw
            # helpful page: https://praw.readthedocs.io/en/latest/code_overview/praw_models.html

# Initialising a connection to Reddit
reddit = praw.Reddit(user_agent='Doing some stuff', client_id='hir_Veg3Rs-Svw', client_secret="giRkbKknUe4Oy4EcoGYA1N_Y0sA",
                    username='', password='')

In [2]:
# General modules and function(s)
import numpy as np
import math
from datetime import datetime
import re
import os
import json
import ijson
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

def utc_time(timestamp):
    '''Changes Unix to UTC time'''
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

## Filter online JSON database of reddit posts in a certain month
Certain templated flairs exist as of March 2019 in r/CasualConversation. However, in previous months, those templated flairs were different or users used their own flairs. Therefore, the approach is to find all the submissions in the JSON database that are made in r/CasualConversations and filter these submissions based on the flairs that can be recognised a topic of discussion. For example, the submissions that have a flair containing the removal reason of the post or a random flair thought up by a user (e.g. 🍕 This subreddit is now sponsored by Domino's), will be filtered out.

### Set-up

In [3]:
# Subreddit to extract submissions from
r_CasualConversation = reddit.subreddit('CasualConversation')

In [4]:
# Month to extract submissions from
current_month = '2017-11'

### Extracting the submissions that are made in r/CasualConversation
The submissinons are extracted from the .xz files on this website for the months 2017-11 until 2018-10: http://files.pushshift.io/reddit/submissions.

**If the JSON files for each month already exist, then skip to the header stating:** _Combining all months into list of JSONs_

In [100]:
all_submissions = []
all_flairs = []
submissions_found = 0

print('Submissions found :')
with open('RS_' + current_month) as json_data:
    for line in json_data:
        json_line = json.loads(line)
        if(json_line['subreddit_id'] != None):
            if(json_line["subreddit"] == 'CasualConversation'):
                all_submissions.append(json_line)
                all_flairs.append(json_line['link_flair_text'])
                submissions_found += 1
                if(json_line['link_flair_text'] != None):
                    print('Count : ' + str(submissions_found) + ' '*(10-len(str(submissions_found))) +
                          'Title : ' + json_line['title'][:40] + ' '*(50-len(str(json_line['title'][:40]))) +
                          'Flair : ' + json_line['link_flair_text']
                         )
                else:
                    print('Count : ' + str(submissions_found) + ' '*(10-len(str(submissions_found))) +
                          'Title : ' + json_line['title'][:40] + ' '*(50-len(str(json_line['title'][:40]))) +
                          'Flair : ' + 'None'
                         )

Submissions found :
Count : 1         Title : Is anyone receiving a lot of trick 'r tr          Flair : abandoned⇢
Count : 2         Title : Would you guys want to be famous? If so,          Flair : None
Count : 3         Title : I have the hair on my face to grow a bea          Flair : None
Count : 4         Title : Do you get called attractive a lot?               Flair : None
Count : 5         Title : AMA?                                              Flair : CAPS⇢
Count : 6         Title : Ask me anything?                                  Flair : None
Count : 7         Title : I tried to make it up to the kids this y          Flair : abandoned⇢
Count : 8         Title : Has anyone *ever* openly read a book in           Flair : book
Count : 9         Title : Does anyone else sometimes feel like run          Flair : None
Count : 10        Title : Anyone else have the flakiest friends on          Flair : weekly thread⇢
Count : 11        Title : Which is better cd or vinyl?             

Count : 178       Title : I'm gonna be moving to Colorado from Neb          Flair : weekly thread⇢
Count : 179       Title : I coreograph absurd fight scenes in my h          Flair : None
Count : 180       Title : Is dwarfism an airborne virus or is it t          Flair : None
Count : 181       Title : I get so nervous taking multiple choice           Flair : None
Count : 182       Title : Hey we're in active lockdown at our scho          Flair : None
Count : 183       Title : Tell me about your 2 week holiday itiner          Flair : None
Count : 184       Title : Playing my iPod while working and have i          Flair : no context⇢
Count : 185       Title : How would you entertain yourself?                 Flair : None
Count : 186       Title : When did you figured out what you wanted          Flair : None
Count : 187       Title : I will argue about anything with anyone.          Flair : None
Count : 188       Title : Would anyone like to play Cards Against           Flair : game
Coun

Count : 355       Title : I moved to a school in the 3rd world cou          Flair : None
Count : 356       Title : I wasn't mugged on Halloween                      Flair : abandoned⇢
Count : 357       Title : What annoys you more than it should?              Flair : None
Count : 358       Title : My nightmares always find a way to be as          Flair : None
Count : 359       Title : Can somebody help me identify a melody/m          Flair : abandoned⇢
Count : 360       Title : Commuting crush                                   Flair : None
Count : 361       Title : Yikes, I am getting a tad reckless on my          Flair : None
Count : 362       Title : I got called in for acting auditions for          Flair : game
Count : 363       Title : I love when people cut me off, speed aro          Flair : None
Count : 364       Title : How to make money quickly without turnin          Flair : None
Count : 365       Title : Made a RP Discord Server                          Flair : banned topic⇢


Count : 532       Title : Has it occurred to anyone?                        Flair : None
Count : 533       Title : Experiences with ex’s after dating?               Flair : weekly thread⇢
Count : 534       Title : Finally got my first job at 18                    Flair : abandoned⇢
Count : 535       Title : What is the most emotional playlist on y          Flair : music
Count : 536       Title : The "I'm a farmer and you can stay the n          Flair : abandoned⇢
Count : 537       Title : Just got rejected by a girl. Need some p          Flair : weekly thread⇢
Count : 538       Title : Dandruff from the guy in front of me fel          Flair : None
Count : 539       Title : How deeply do your online relationships           Flair : None
Count : 540       Title : Update: Just got back from what turns ou          Flair : weekly thread⇢
Count : 541       Title : I hate it when I'm trying to sing a song          Flair : music
Count : 542       Title : Working over night keep me company!     

Count : 711       Title : Does anyone else hate it when a game tak          Flair : game
Count : 712       Title : One of those crushes that isn’t a crush?          Flair : None
Count : 713       Title : Little at the worst point in my life, ho          Flair : None
Count : 714       Title : I have no idea what its like to be 'with          Flair : None
Count : 715       Title : Teachers' words can hurt.                         Flair : None
Count : 716       Title : Starting to feel really, really lonely            Flair : None
Count : 717       Title : I'm about to eat candy that's been disco          Flair : None
Count : 718       Title : I wonder how many miles I've scrolled wi          Flair : no context⇢
Count : 719       Title : how do you r/casualConversation peeps ma          Flair : None
Count : 720       Title : Looking to exchange pics with a girl sna          Flair : no context⇢
Count : 721       Title : I just want to talk to a girl                     Flair : None
Count :

...

Count : 6420      Title : Looking for some video games to play, bu          Flair : game
Count : 6421      Title : Keep my karma at exactly 420                      Flair : no context⇢
Count : 6422      Title : How would you comfort your girlfriend if          Flair : no context⇢
Count : 6423      Title : When my traditional mom drives                    Flair : None
Count : 6424      Title : Yesterday, I discovered the story of Ran          Flair : None
Count : 6425      Title : Smashed my finger in door. Always had a           Flair : no context⇢
Count : 6426      Title : I just broke up with my girlfriend. I ne          Flair : no context⇢
Count : 6427      Title : It's gotten to the point where I'm nervo          Flair : None
Count : 6428      Title : If you're afraid of spiders then it is b          Flair : None
Count : 6429      Title : Good day, I love you. :)                          Flair : no context⇢
Count : 6430      Title : Butch Hartman liked my tweet.                    

Count : 6596      Title : I sometimes hear noises that nobody else          Flair : None
Count : 6597      Title : Trying to stay awake                              Flair : None
Count : 6598      Title : We have started to diminish the acceptab          Flair : None
Count : 6599      Title : My boyfriend broke up with me last night          Flair : weekly thread⇢
Count : 6600      Title : The person I love left me because they n          Flair : weekly thread⇢
Count : 6601      Title : I’m sitting in math class bored as hell           Flair : no context⇢
Count : 6602      Title : TIL that digestive system stores more li          Flair : no context⇢
Count : 6603      Title : Losing it, let's talk                             Flair : weekly thread⇢
Count : 6604      Title : 5 q's going up... on a tuesday.                   Flair : None
Count : 6605      Title : My abuser works at Victoria's Secret              Flair : None
Count : 6606      Title : I have depression, my friends all live f

Count : 6772      Title : What's something that you love but don't          Flair : None
Count : 6773      Title : Sweet relationship without label is Toxi          Flair : no context⇢
Count : 6774      Title : Let's play a game: Which is worse?                Flair : game
Count : 6775      Title : Sweet relationship without label is toxi          Flair : no context⇢
Count : 6776      Title : About 3 weeks ago the Louder With Crowde          Flair : abandoned⇢
Count : 6777      Title : My guinea pig died :&lt;                          Flair : banned topic⇢
Count : 6778      Title : My roommate and his girlfriend just requ          Flair : no context⇢
Count : 6779      Title : R/dataiscreepy is a real bummer of a sub          Flair : no context⇢
Count : 6780      Title : Roommates and friend don't know im home           Flair : None
Count : 6781      Title : I'm basically a robot, aren't we all?             Flair : abandoned⇢
Count : 6782      Title : What themed trivia could you win, h

Count : 6948      Title : Let's play truth or dare! (Possibly NSFW          Flair : game
Count : 6949      Title : I feel that if I was less self aware I w          Flair : no context⇢
Count : 6950      Title : Would you be a jedi or a sith?                    Flair : None
Count : 6951      Title : I'm so uptight, if a post is upvoted to           Flair : no context⇢
Count : 6952      Title : I'm so uptight, if a post is sitting on           Flair : abandoned⇢
Count : 6953      Title : I just ate a full bag of chips, finishin          Flair : no context⇢
Count : 6954      Title : The screen on my phone shattered, and I'          Flair : None
Count : 6955      Title : As of today I’m quitting drinking soda            Flair : None
Count : 6956      Title : Last night, I was an inch close to being          Flair : weekly thread⇢
Count : 6957      Title : I just ate a full bag of chips, am finis          Flair : None
Count : 6958      Title : What’s for dinner?                             

Count : 7123      Title : Advice for ordering shirts on the intern          Flair : None
Count : 7124      Title : You're not my boss                                Flair : None
Count : 7125      Title : Today, I am presenting as a woman for th          Flair : None
Count : 7126      Title : No classes today, waiting for work.               Flair : banned topic⇢
Count : 7127      Title : Subreddit rules                                   Flair : None
Count : 7128      Title : Did I Just Ruin This Girl's Life on Chat          Flair : None
Count : 7129      Title : This rap about social media. Any thought          Flair : abandoned⇢
Count : 7130      Title : Started reading iron Druid book series,           Flair : book
Count : 7131      Title : I've got a school project coming up soon          Flair : None
Count : 7132      Title : I can't stand when SJW's refuse to ident          Flair : None
Count : 7133      Title : Thought I made a guys day, may have ruin          Flair : abandoned⇢


### Opening and saving submissions
The following cells save and open the JSON files that contain the submissions for the CasualConversation subreddit for a current month. The subreddit and current_month were stated in the _Set-up_ before.

In [5]:
# Saving is commented out to prevent accidental saving
# with open('submissions_CasualConversation_' + current_month + '.json', 'w+') as save_file:
#     for submission in all_submissions:
#         save_file.write("%s\n" % json.dumps(submission))

NameError: name 'all_submissions' is not defined

In [102]:
saved_submissions = []
saved_flairs = []

with open('submissions_CasualConversation_' + current_month + '.json', 'r') as save_file:
    saved_lines = save_file.readlines()
    for line in saved_lines:
        json_line = json.loads(line)
        saved_submissions.append(json_line)
        saved_flairs.append(json_line['link_flair_text'])

In [103]:
save_file.close()

### Combining all months to list of JSONs
The months included go from 2017-11 until 2018-10

In [271]:
all_months_submissions = []
all_months_flairs = []
months = ['2017-11', '2017-12', '2018-01',
          '2018-02', '2018-03', '2018-04',
          '2018-05', '2018-06', '2018-07',
          '2018-08', '2018-09', '2018-10']
for month in months:
    with open('submissions_CasualConversation_' + month + '.json', 'r') as save_file:
        lines = save_file.readlines()
        for line in lines:
            json_line = json.loads(line)
            all_months_submissions.append(json_line)
            all_months_flairs.append(json_line['link_flair_text'])
    save_file.close()

In [272]:
len(all_months_submissions)

83354

### Filtering based on flair and saving filtered data

In [273]:
# Showing flair templates requires the reddit instance to have a logged in Reddit account

flair_templates = list(r_CasualConversation.flair.link_templates)

flair_ids = []
for flair_template in flair_templates:
    flair_ids.append(flair_templates)
    print(flair_template['text'])

APIException: USER_REQUIRED: 'Please log in to do that.'

In [274]:
# Printing all the flairs that currently exist among the submissions
all_months_flairs_set = set(all_months_flairs)
flair_column = []
count_column = []

for flair in all_months_flairs_set:
    if(flair != None):
        flair_column.append(flair)
        count_column.append(all_months_flairs.count(flair))
#         print(flair + ' ' + str(all_months_flairs.count(flair)))

flair_count_df = pd.DataFrame(data={'flair':flair_column, 'count':count_column})
flair_count_df = flair_count_df.sort_values(by='count',ascending=False)
flair_count_df.iloc[:40,:]

,flair,count
56,no context⇢,16933
46,banned topic⇢,8416
4,abandoned⇢,5291
12,weekly thread⇢,1660
26,Music,813
31,music,504
58,Movies &amp; Shows,500
50,Gaming,480
15,game,207
60,no context,198


In [275]:
flair_count_df.iloc[40:,:]

,flair,count
32,banned topic⇢ respect⇢,4
18,abandoned⇢ weekly thread⇢,4
11,banned topic⇢ privacy⇢,3
14,weekly thread⇢ privacy⇢,3
68,:star: neat,2
48,megathread,2
53,Removed,2
28,podcast,2
22,banned topic⇢ vitriolic⇢,2
75,Megathread,2


In [276]:
# Grouping flairs together
music_group = ['Music', 'music']
movie_show_group = ['Movies &amp; Shows', 'movie', 'movies &amp; shows',
                    'show', 'movie or show', 'movie/show', 'Movies',
                    'Television', ':film: movies &amp; shows', 'Shows',
                    'Movies &amp; Theatre (&amp; Books?)']
gaming_group = ['Gaming', 'game', 'gaming']
book_group = ['Book', 'book']
neat_group = ['Neat', 'neat', ':star: neat', 'neat - party time!', 'Neat r/all']

for submission in all_months_submissions:
    if(music_group.count(submission['link_flair_text']) > 0):
        submission['link_flair_text'] = 'Music'
    elif(movie_show_group.count(submission['link_flair_text']) > 0):
        submission['link_flair_text'] = 'Movies &amp; Shows'
    elif(gaming_group.count(submission['link_flair_text']) > 0):
        submission['link_flair_text'] = 'Gaming'
    elif(book_group.count(submission['link_flair_text']) > 0):
        submission['link_flair_text'] = 'Book'
    elif(neat_group.count(submission['link_flair_text']) > 0):
        submission['link_flair_text'] = 'Neat'
    else:
        submission['link_flair_text'] = None

In [277]:
# Printing the amount of submissions with a flair
# Note that in the previous cell the flairs were reduced to 5 groups and will otherwise be 'None'
counter = 0
for submission in all_months_submissions:
    if(submission['link_flair_text'] != None):
        counter += 1
print(counter)

3340


In [278]:
# Removing all submissions which contents were removed or deleted before they were extracted from Reddit
# and put in the database that is used for this Notebook, and removing all submissions that don't have flairs.
flair_filtered_submissions = []
flair_filtered_flairs = []

for submission in all_months_submissions:
    if(submission['link_flair_text'] != None and submission['selftext'] != '[removed]' and submission['selftext'] != '[deleted]'):
        flair_filtered_submissions.append(submission)
        flair_filtered_flairs.append(submission['link_flair_text'])

In [283]:
# Printing the amount of submissions for each flair group for the remaining submissions.
flair_filtered_flairs_set = set(flair_filtered_flairs)
flair_column = []
count_column = []

for flair in flair_filtered_flairs_set:
    if(flair != None):
        flair_column.append(flair)
        count_column.append(flair_filtered_flairs.count(flair))
#         print(flair + ' ' + str(all_months_flairs.count(flair)))

flair_count_df = pd.DataFrame(data={'flair':flair_column, 'count':count_column})
flair_count_df = flair_count_df.sort_values(by='count',ascending=False)
flair_count_df

,flair,count
2,Music,1039
4,Movies &amp; Shows,645
1,Gaming,615
3,Book,171
0,Neat,140


In [284]:
len(flair_filtered_submissions)

2610

In [286]:
# Here it can be manually inspected that each submission is now filtered 
# to have one of the hand-picked flairs and has not been removed or deleted.
for submission in flair_filtered_submissions:
    print(submission['selftext'][:9], submission['id'], submission['link_flair_text'])

Your thou 7a0chi Book
And yes,  7a0lcd Movies &amp; Shows
Starting  7a0mg1 Gaming
That is,  7a0v73 Movies &amp; Shows
Starting  7a16ye Gaming
There’s s 7a1m5t Gaming
Let's hea 7a1sc0 Movies &amp; Shows
The movie 7a1x90 Movies &amp; Shows
I've been 7a26e1 Gaming
What movi 7a3ql5 Movies &amp; Shows
What are  7a4570 Music
I love he 7a4f5m Music
I started 7a4uan Gaming
New York  7a4v7e Book
I'm lonel 7a5l46 Gaming
It clearl 7a5nnw Gaming
How's eve 7a7zo0 Music
I'm just  7a8ld3 Movies &amp; Shows
This game 7a8r62 Gaming
I really  7a96mo Movies &amp; Shows
Back in h 7a9oro Music
It doesn' 7aa8am Music
I've only 7aa8bw Music
I'm so af 7ab4iw Gaming
I don't k 7abfj5 Gaming
It's cold 7acaza Music
So in the 7aeau3 Music
That Girl 7af4y8 Movies &amp; Shows
Those who 7agfka Music
Post your 7ahe4i Music
Just me?  7ahmo8 Music
No countr 7ai21i Music
I started 7aifwo Gaming
And what' 7ajcij Music
https://i 7ajoxz Movies &amp; Shows
For me, h 7an27f Book
Note: Thi 7an454 Movies &amp; Shows
Like in t 7

Everytime 7px0ts Music
So I'll b 7pxe33 Music
Just as o 7pyoht Gaming
Okay so f 7pz3jr Gaming
[This Yea 7q095f Music


My love 7q1mkf Music
August an 7q1ow6 Music
I'm 22, c 7q1yvu Gaming
I could s 7q20rf Movies &amp; Shows
If you wa 7q2507 Gaming
My main s 7q2la7 Movies &amp; Shows
Some of t 7q43kn Movies &amp; Shows
So me and 7q4u0r Movies &amp; Shows
I just st 7q6f1a Movies &amp; Shows
I'm a hug 7q6kpm Book
A quote t 7q7f5q Movies &amp; Shows
I've neve 7q817t Music
Mine is ( 7q8dfr Music
I know th 7q96rr Movies &amp; Shows
Hey guys, 7q9n1w Gaming
I was pla 7qaw6p Gaming
So my tas 7qc2q0 Music
So i woke 7qcc2b Movies &amp; Shows
And Provi 7qd0jy Music
With Seth 7qd34i Movies &amp; Shows
Hey guys, 7qdr8j Music
Having a  7qe2dr Movies &amp; Shows
So I was  7qg7xs Gaming
The title 7qgrrp Music
A few mon 7qgu7p Music
Hi o/

Du 7qhf0q Gaming
At first, 7qhx2a Neat
I don't k 7qijuj Movies &amp; Shows
I'm curre 7qisu4 Gaming
And every 7qjc2t Movies &amp; Shows
I only ha 7qjs2b Gaming
Zombies,

Does anyo 87om9j Music
It's just 87p1mb Music
Is the fa 87pm0v Gaming
I love ga 87xo0d Gaming
^^^For ^^ 87y88p Movies &amp; Shows
So, I jus 87zcyy Movies &amp; Shows
**Quick N 880q9e Movies &amp; Shows
This week 881vds Music
I don't k 886ng4 Music
I don’t k 88737t Music
So basica 8879u2 Gaming
I've been 887liu Music
Maybe you 8888xl Music
Never did 888j7y Gaming
[Here it  888zqt Music
I honestl 8890t4 Movies &amp; Shows
Hi, I was 88a50s Music
A long aw 88a6ce Gaming
Like the  88b122 Book
I've been 88cz92 Music
If not yo 88du8c Gaming
Hey all,
 88eqk8 Music
I know it 88fdnh Music
Whenever  88fwhi Gaming
Basically 88g3e8 Music
Ok so I c 88hmmk Music
I've only 88ji8p Music
Anyone el 88kgvn Gaming
Ok a litt 88lluu Movies &amp; Shows
My curren 88lnxz Music
I’m an al 88n2in Music
For the l 88qqnz Neat
The PSD i 88rs6k Movies &amp; Shows
my dad us 88skgx Music
So I’m re 88swss Music
Hi guys!  88swzy Music
I loved t 88ur9u Movies &amp; Shows
So Casual 88vb6o Gaming
I've been 88vn9m Gaming
I'm 

For me, C 972nbv Movies &amp; Shows
Me and my 973sm3 Movies &amp; Shows
I was pla 975fvx Gaming
I am in m 975qmy Movies &amp; Shows
I'm 28 ye 976lih Gaming
I'm used  976uve Movies &amp; Shows
I recentl 9774vi Music
I sometim 9777jq Music
Hey every 97bh5r Gaming
This ques 97dp41 Movies &amp; Shows
"Trailer  97hnns Music
Imagine m 97lgpm Music
I’m reall 97mblv Music
I'll star 97ndcs Music
I collect 97nl3a Gaming
Probably  97nnn5 Movies &amp; Shows
I tried l 97nsmj Book
For the p 97qy5w Movies &amp; Shows
So don't  97sb0f Movies &amp; Shows
A couple  97t505 Neat
a bit ago 97zaxn Gaming
I wasn't  97zrvr Music
For me pe 98087k Gaming
I mean ev 981kaq Movies &amp; Shows
Here are  9835i3 Music
I normall 983bda Movies &amp; Shows
Might be  984c7z Movies &amp; Shows
Like, hol 984tdv Movies &amp; Shows
okay, I d 986ftq Gaming
Trying to 986gd1 Movies &amp; Shows
I know mo 988edc Music
I must co 98buko Music
1. What's 98h1ru Music
You ever  98hei3 Gaming
George W  98hi37 Neat
I’m hopin 98igco Book

## Adding comment information
In this section, the comments per submission are extracted from reddit and two features will be created that are appended as items to the JSON of each submission:

> A time series for the comment influx for time period 't' with time steps 'dt'. E.g. the amount of comments made every 10 minutes for the 2 hours after a submission has been made. The timestamp of the comments is used to create this time series.

> The plain text of the comments. For now, only the top-level comments are included in this.

In [287]:
def comment_feature_extraction(t,dt):
    '''Extracts features from the comments of a submission.
    
    Presume the scenario that the comments are being observed for 't'seconds. 
    Every 'dt' seconds, the amount of comments that are now underneath the
    post are noted down. The text of the toplevel comments (those comments
    that directly address the post) is saved after 't' seconds.
    
    ============================= =============================================
    Parameters                    Description
    ============================= =============================================
    "t"                           Amount of seconds for whichthe post is
                                  observed.
    "dt"                          Amount of seconds between each observation.
    ============================= =============================================
    Returns                       Description
    ============================= =============================================
    "comment_influxes"            List of observed comments per 'dt' seconds
                                  for all submissions. (list of list)
    "comment_toplevel_text_total" List for which each element is the text of a
                                  toplevel comment for all submissions. (list 
                                  of list)
    ============================= =============================================
    
    '''
    comment_influxes = []
    comment_toplevel_text_total = []
    for submission in tqdm(flair_filtered_submissions):
        # Get submission from reddit using PRAW and get its timestamp.
        reddit_submission = reddit.submission(submission['id'])
        submission_timestamp = reddit_submission.created_utc
        
        # Get the submission's top-level comments without the 'MoreComments'.
        reddit_submission.comments.replace_more(limit=0)
        top_level_comments = reddit_submission.comments
        
        # These will be the list containing the timedifferences of all the comments with the submission,
        # and the list containing all the text of the top-level comments.
        comment_timedifferences = []
        comment_toplevel_text = []
        
        # Check for every top-level comment whether it is within the timeframe:
        # - if so, then add the body of that comment to the list of text that includes each
        #   top-level comment's body up within timeframe 't'; also add the timedifferences for
        #   the replies to the list of comment time differences, as these can be within timeframe
        #   't'.
        # - if not, then don't check the replies.
        for top_level_comment in top_level_comments:
            comment_timedifferences.append(top_level_comment.created_utc - submission_timestamp)
            if(comment_timedifferences[-1] < t):
                comment_toplevel_text.append(top_level_comment.body)
                for reply in top_level_comment.replies.list():
                    comment_timedifferences.append(reply.created_utc - submission_timestamp)
        
        # Sort the timedifferences and track the sum of comments for each timestep 'dt'
        # within timeframe 't'
        comment_timedifferences.sort()
        comment_sum = [0] * (math.floor(t/dt))
        for i, comment_timedifference in enumerate(comment_timedifferences):
            if(comment_timedifference >= t): break
            comment_sum[math.floor(comment_timedifference/dt)] = i + 1
        for i in range(1,len(comment_sum)):
            if(comment_sum[i] == 0):
                comment_sum[i] = comment_sum[i-1]
        
        # Append the submission's influx of comments and top-level comment text
        # to these lists, so that these features can be created for all submissions
        comment_influxes.append(comment_sum)
        comment_toplevel_text_total.append(comment_toplevel_text)
    return comment_influxes, comment_toplevel_text_total

In [288]:
# Clearing the lists that should contain the influx of comments for each submission,
# and the text of the toplevel comments for each submission
comment_influxes = []
comment_toplevel_text = []
comment_influxes, comment_toplevel_text = comment_feature_extraction(3600,150)

In [293]:
# Flatten the list of comment_toplevel_text. Each element is now no longer a
# list with the text for each comment as an indivual element in that list, but
# instead the comments for a submission are concatenated into one element per
# submission.
for i in range(len(flair_filtered_submissions)):
    flair_filtered_submissions[i]['comment_influx'] = comment_influxes[i]
    all_comment_toplevel_text = ''
    for text in comment_toplevel_text[i]:
        all_comment_toplevel_text += ' \n ' + text
    flair_filtered_submissions[i]['comment_toplevel_text'] = all_comment_toplevel_text

## Saving submissions and opening them

### Saving submissions

In [295]:
save_name = 'submissions_CasualConversation_filtered_with_comment_data'
save_submissions = flair_filtered_submissions
with open(save_name + '.json', 'w+') as save_file:
    for submission in save_submissions:
        save_file.write("%s\n" % json.dumps(submission))

### Submission inspection

In [191]:
saved_submissions = []
saved_flairs = []

with open(save_name + '.json', 'r') as save_file:
    saved_lines = save_file.readlines()
    for line in saved_lines:
        json_line = json.loads(line)
        saved_submissions.append(json_line)
        saved_flairs.append(json_line['link_flair_text'])

In [ ]:
for i in range(100):
    print(saved_submissions[i]['title'],'\n',saved_submissions[i]['selftext'])
    print(saved_submissions[i]['comment_influx'])
    print('\n',saved_submissions[i])
    print('------')

Here can be more cells that look into the submissions file opened above.